![title](securly-banner2.jpg)

In [3]:
!conda install -qy scikit-learn scipy matplotlib
!pip install -q -U python-dotenv
!pip install -q -U datasets # The version in conda is broken
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U vllm

In [4]:
from vllm import LLM, SamplingParams

ImportError: libcudart.so.12: cannot open shared object file: No such file or directory